# Restricted Boltzman Machine (Movie Recommender System)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

*Importing dataset*

In [2]:
movies = pd.read_csv('dataset/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
users = pd.read_csv('dataset/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [4]:
ratings = pd.read_csv('dataset/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

*Creating the training and test set*

In [5]:
training_set = pd.read_csv('dataset/ml-100k/u1.base', delimiter='\t')

In [6]:
training_set = np.array(training_set, dtype='int')

In [7]:
test_set = np.array(pd.read_csv('dataset/ml-100k/u1.test', delimiter='\t'), dtype='int')

*Finding total number of movies and users*

In [8]:
total_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
total_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

*Converting data into movies as columns and users as rows*

In [9]:
def convert(data):
  new_data = []
  for id_users in range(1, total_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(total_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

*Converting data into torch tensors*

In [10]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

*Converting ratings in a binary systen of 1 (Liked) and 0 (Disliked)*

In [11]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

*Creating the RBM architecture and training it*

In [12]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

In [13]:
visible_features = len(training_set[0])
hidden_features = 10000
batch_size = 32

In [14]:
rbm = RBM(visible_features, hidden_features)

In [15]:
epochs = 25
k_steps = 20
for epoch in range(1, epochs + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, total_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(20):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0])).item()
    s += 1.
  print('epoch:', epoch, 'loss:', train_loss / s)

epoch: 1 loss: 0.3168873807479595
epoch: 2 loss: 0.2772014896417486
epoch: 3 loss: 0.2580833691975166
epoch: 4 loss: 0.24644158780574799
epoch: 5 loss: 0.24525945710724797
epoch: 6 loss: 0.24520620138480745
epoch: 7 loss: 0.24038453708434926
epoch: 8 loss: 0.2405709146425642
epoch: 9 loss: 0.23899131550871092
epoch: 10 loss: 0.24376010124025674
epoch: 11 loss: 0.24162490563146
epoch: 12 loss: 0.23666475507719764
epoch: 13 loss: 0.23760996090954747
epoch: 14 loss: 0.23891742784401465
epoch: 15 loss: 0.23967861307078395
epoch: 16 loss: 0.23851921332293544
epoch: 17 loss: 0.2371861220433794
epoch: 18 loss: 0.23750380452336936
epoch: 19 loss: 0.23664871012342387
epoch: 20 loss: 0.23758633486155806
epoch: 21 loss: 0.23911487388199773
epoch: 22 loss: 0.23928865173767352
epoch: 23 loss: 0.23805844372716442
epoch: 24 loss: 0.23928616478525358
epoch: 25 loss: 0.23832169637597841


*Testing RBM*

In [16]:
test_loss = 0
s = 0.
for id_user in range(total_users):
    v = training_set[id_user:id_user + 1]
    vt = test_set[id_user:id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0])).item()
        s += 1.
print('test loss:', test_loss / s)

test loss: 0.24534429945782119
